## Telemetry Logging with AutoGen 

AutoGen offers some utilities to log telemetry data, useful for debugging and performance analysis. This notebook demonstrates how to use these utilities. 

In general, users can initiate logging by calling `autogen.telemetry.start_logging` and stop logging by calling `autogen.telemetry.stop_logging`, and get logged data by calling `autogen.telemetry.get_log`.

In [1]:
import os
import json
import autogen
from autogen import AssistantAgent, UserProxyAgent

# start logging telemetry data
telemetry_session_id = autogen.telemetry.start_logging(dbname="telemetry.db")
print("Telemetry session ID: " + str(telemetry_session_id))

## Create an Agent Workflow and Run it
llm_config = {
    "config_list": [
        {
            "model": "gpt-4-turbo-preview",
            "api_key": os.environ["OPENAI_API_KEY"],  # Note, add your own API key here
        }
    ],
    "temperature": 0.9,
}

## start_logging
assistant = AssistantAgent(name="assistant", llm_config=llm_config)
user_proxy = UserProxyAgent(
    name="user_proxy",
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "TERMINATE" in msg["content"],
)

user_proxy.initiate_chat(
    assistant, message="What is the height of the Eiffel Tower? Only respond with the answer and terminate"
)
autogen.telemetry.stop_logging()

Telemetry session ID: None
user_proxy (to assistant):

What is the height of the Eiffel Tower? Only respond with the answer and terminate

--------------------------------------------------------------------------------
assistant (to user_proxy):

The height of the Eiffel Tower is 330 meters (1,083 feet).

TERMINATE

--------------------------------------------------------------------------------


## Getting Data from the SQL Lite Database 

We can get data from the database by calling `autogen.telemetry.get_log`. This function returns a pandas dataframe with the logged data. 



In [7]:
def str_to_dict(s):
    return json.loads(s)


telemetry_df = autogen.telemetry.get_log(table="chat_completions")
telemetry_df["total_tokens"] = telemetry_df.apply(
    lambda row: str_to_dict(row["response"])["usage"]["total_tokens"], axis=1
)


telemetry_df

,id,invocation_id,client_id,wrapper_id,session_id,request,response,is_cached,cost,start_time,end_time,total_tokens
0,1,4f78a8e1-1849-475d-8b27-16ef5ac19965,139914219757312,139914219756016,2430593b-954a-491a-962e-ffd7eba0101f,"{""messages"": [{""content"": ""You are a helpful A...","{\n ""id"": ""chatcmpl-8nBekDzT58BHYVz8KeVcgtv...",1,0.0,2024-01-31 21:07:08.154565,2024-01-31 21:07:08.154768,511
1,2,ecb59f50-85a9-4009-817f-da2a9b2ccd39,140552952983904,140552953221120,aac4f82e-3363-45e6-a2f1-f350ecad23fd,"{""messages"": [{""content"": ""You are a helpful A...","{\n ""id"": ""chatcmpl-8nBekDzT58BHYVz8KeVcgtv...",1,0.0,2024-01-31 21:08:52.132000,2024-01-31 21:08:52.132223,511
2,3,8ebfbb71-bc19-4aa9-ac44-298ac8f26606,140564553711328,140564613259008,6d0728c1-9b7c-43e8-941c-45e3bae0f552,"{""messages"": [{""content"": ""You are a helpful A...","{\n ""id"": ""chatcmpl-8nBekDzT58BHYVz8KeVcgtv...",1,0.0,2024-01-31 21:10:00.956998,2024-01-31 21:10:00.957146,511


## Computing Cost 

One use case of telemetry data is to compute the cost of a session.

In [21]:
# total tokens for all sessions .. sum totoal tokens for all sessions
total_tokens = telemetry_df["total_tokens"].sum()
# total tokens for specific session telemetry_session_id
total_tokens = telemetry_df[telemetry_df["session_id"] == telemetry_session_id]["total_tokens"].sum()

print("Total tokens for all sessions: " + str(total_tokens))
print("Total tokens for session " + str(telemetry_session_id) + ": " + str(total_tokens))

Total Cost: 0.0
Total Tokens: 511
